In [2]:
!pip install git+https://github.com/EleutherAI/sae.git
!pip install datasets transformers torch


  Cloning https://github.com/EleutherAI/sae.git to /tmp/pip-req-build-c8oug7k9
  Running command git clone --filter=blob:none --quiet https://github.com/EleutherAI/sae.git /tmp/pip-req-build-c8oug7k9
  Resolved https://github.com/EleutherAI/sae.git to commit ce1fe2b1b8f0fe117f42249ae48c9fb4f39b2e5a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

from sae import SaeConfig, SaeTrainer, TrainConfig
from sae.data import chunk_and_tokenize

# Set up the model and dataset
MODEL = "EleutherAI/pythia-160m"
dataset = load_dataset(
    "togethercomputer/RedPajama-Data-1T-Sample",
    split="train",
    trust_remote_code=True
)

# Initialize tokenizer and tokenize the dataset
tokenizer = AutoTokenizer.from_pretrained(MODEL)
tokenized = chunk_and_tokenize(dataset, tokenizer)

# Load the model
gpt = AutoModelForCausalLM.from_pretrained(
    MODEL,
    device_map={"": "cuda"},
    torch_dtype=torch.bfloat16,
)

# Configure the SAE training
cfg = TrainConfig(
    SaeConfig(gpt.config.hidden_size),
    batch_size=16,
    # grad_acc_steps=4,
    # lr=1e-3,
    # save_every=1000,
    # run_name="sae_training_run"
)

# Initialize the trainer
trainer = SaeTrainer(cfg, tokenized, gpt)

In [ ]:
# Start training
trainer.fit()